DESCRIPTION

One of the leading retail stores in the US, Walmart, would like to predict the sales and demand accurately. There are certain events and holidays which impact sales on each day. There are sales data available for 45 stores of Walmart. The business is facing a challenge due to unforeseen demands and runs out of stock some times, due to the inappropriate machine learning algorithm. An 

ideal ML algorithm will predict demand accurately and ingest factors like economic conditions including CPI, Unemployment Index, etc.

Walmart runs several promotional markdown events throughout the year. These markdowns precede prominent holidays, the four largest of all, which are the Super Bowl, Labour Day, Thanksgiving, and Christmas. The weeks including these holidays are weighted five times higher in the evaluation than non-holiday weeks. Part of the challenge presented by this competition is modeling the effects of markdowns on these holiday weeks in the absence of complete/ideal historical data. Historical sales data for 45 Walmart stores located in different regions are available.

Holiday Events

Super Bowl: 12-Feb-10, 11-Feb-11, 10-Feb-12, 8-Feb-13
Labour Day: 10-Sep-10, 9-Sep-11, 7-Sep-12, 6-Sep-13
Thanksgiving: 26-Nov-10, 25-Nov-11, 23-Nov-12, 29-Nov-13
Christmas: 31-Dec-10, 30-Dec-11, 28-Dec-12, 27-Dec-13


Analysis Tasks

Basic Statistics tasks

Which store has maximum sales

Which store has maximum standard deviation i.e., the sales vary a lot. Also, find out the coefficient of mean to standard deviation

Which store/s has good quarterly growth rate in Q3’2012

Some holidays have a negative impact on sales. Find out holidays which have higher sales than the mean sales in non-holiday season for all stores together

Provide a monthly and semester view of sales in units and give insights

Statistical Model

For Store 1 – Build  prediction models to forecast demand

Linear Regression – Utilize variables like date and restructure dates as 1 for 5 Feb 2010 (starting from the earliest date in order). Hypothesize if CPI, unemployment, and fuel price have any impact on sales.

Change dates into days by creating new variable.

Select the model which gives best accuracy.

In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv("Walmart_Store_sales.csv")

In [3]:
data.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,05-02-2010,1643690.90,0,42.31,2.572,211.096358,8.106
1,1,12-02-2010,1641957.44,1,38.51,2.548,211.242170,8.106
2,1,19-02-2010,1611968.17,0,39.93,2.514,211.289143,8.106
3,1,26-02-2010,1409727.59,0,46.63,2.561,211.319643,8.106
4,1,05-03-2010,1554806.68,0,46.50,2.625,211.350143,8.106


In [4]:
data.dtypes

Store             int64
Date             object
Weekly_Sales    float64
Holiday_Flag      int64
Temperature     float64
Fuel_Price      float64
CPI             float64
Unemployment    float64
dtype: object

In [5]:
# find the store with max sales
sum_sales = data.groupby(['Store'])['Weekly_Sales'].sum().reset_index()
max_sale = sum_sales['Weekly_Sales'].max()
sum_sales[sum_sales['Weekly_Sales']==max_sale]

,Store,Weekly_Sales
19,20,3.013978e+08


In [6]:
# find the store with max sd
sd_sales = data.groupby(['Store'])['Weekly_Sales'].std().reset_index()
max_sd_sale = sd_sales['Weekly_Sales'].max() 
sd_sales[sd_sales['Weekly_Sales']==max_sd_sale]

,Store,Weekly_Sales
13,14,317569.949476


In [7]:
# find coeff of mean to std deviation
mean_sales = data.groupby(['Store'])['Weekly_Sales'].mean().reset_index()
sd_mean_sales = pd.DataFrame()
sd_mean_sales['Store'] = sd_sales['Store']
sd_mean_sales['sd_sales'] = sd_sales['Weekly_Sales']
sd_mean_sales['mean_sales'] = mean_sales['Weekly_Sales']
sd_mean_sales['CV_sales'] = sd_mean_sales['sd_sales']/sd_mean_sales['mean_sales']
sd_mean_sales[sd_mean_sales['CV_sales']==sd_mean_sales['CV_sales'].max()]

,Store,sd_sales,mean_sales,CV_sales
34,35,211243.457791,919724.97958,0.229681


In [8]:
#Which store/s has good quarterly growth rate in Q3’2012
data['Date'] = pd.to_datetime(data.Date.astype(str), format='%d-%m-%Y') # convert date to datetime
data['quarter'] = data.Date.dt.year.astype(str) + '.' + data.Date.dt.quarter.astype(str) 
quarter_sales = data.groupby(['Store','quarter'])['Weekly_Sales'].sum().reset_index() # calculate sum of sales group by store and quarter
sales_2_2012 = quarter_sales[quarter_sales.quarter.eq("2012.2")] # fetch 2nd quarter of 2012
sales_3_2012 = quarter_sales[quarter_sales.quarter.eq("2012.3")] # fetch 3rd quarter of 2012
sales_2_2012.index = range(45) # set index
sales_3_2012.index = range(45) # set index
sales_2_3_2012 = pd.DataFrame() # create a new df with quarter 2 and quarter 3 sales 
sales_2_3_2012['Store'] = sales_2_2012['Store']
sales_2_3_2012['2_quarter_sale'] = sales_2_2012['Weekly_Sales']
sales_2_3_2012['3_quarter_sale'] = sales_3_2012['Weekly_Sales']
sales_2_3_2012['q3_growth'] = (sales_2_3_2012['3_quarter_sale']-sales_2_3_2012['2_quarter_sale'])/(sales_2_3_2012['2_quarter_sale'])
max_growth = sales_2_3_2012['q3_growth'].max()
sales_2_3_2012[sales_2_3_2012['q3_growth']==max_growth]

,Store,2_quarter_sale,3_quarter_sale,q3_growth
6,7,7290859.27,8262787.39,0.133308


In [9]:
# Find out holidays which have higher sales than the mean sales in non-holiday season for all stores together
non_holiday_sales = data[data.Holiday_Flag.eq(0)]
mean_non_holiday_sales = non_holiday_sales['Weekly_Sales'].mean()
holiday_sales = data[data.Holiday_Flag.eq(1)]
holiday_sales = holiday_sales.groupby(['Date'])['Weekly_Sales'].sum().reset_index()
holiday_sales[holiday_sales['Weekly_Sales']>mean_non_holiday_sales]

,Date,Weekly_Sales
0,2010-02-12,48336677.63
1,2010-09-10,45634397.84
2,2010-11-26,65821003.24
3,2010-12-31,40432519.00
4,2011-02-11,47336192.79
5,2011-09-09,46763227.53
6,2011-11-25,66593605.26
7,2011-12-30,46042461.04
8,2012-02-10,50009407.92
9,2012-09-07,48330059.31


In [10]:
#Provide a monthly and semester view of sales in units and give insights
data['month'] = data.Date.dt.year.astype(str) + '.' + data.Date.dt.month.astype(str) 
data['sem']= data.Date.dt.year.astype(str) + '.'+ np.where(data.Date.dt.quarter.gt(2),2,1).astype(str)
monthly_sales = data.groupby(['month'])['Weekly_Sales'].sum().reset_index()
sem_sales = data.groupby(['sem'])['Weekly_Sales'].sum().reset_index()
print(monthly_sales[monthly_sales['Weekly_Sales']==monthly_sales['Weekly_Sales'].max()])
print()
print(sem_sales[sem_sales['Weekly_Sales']==sem_sales['Weekly_Sales'].max()])

     month  Weekly_Sales
2  2010.12  2.887605e+08

      sem  Weekly_Sales
3  2011.2  1.320860e+09


For Store 1 – Build prediction models to forecast demand

Linear Regression – Utilize variables like date and restructure dates as 1 for 5 Feb 2010 
(starting from the earliest date in order). Hypothesize if CPI, unemployment, and fuel price have any impact on sales.

Change dates into days by creating new variable.

In [11]:
data.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,quarter,month,sem
0,1,2010-02-05,1643690.90,0,42.31,2.572,211.096358,8.106,2010.1,2010.2,2010.1
1,1,2010-02-12,1641957.44,1,38.51,2.548,211.242170,8.106,2010.1,2010.2,2010.1
2,1,2010-02-19,1611968.17,0,39.93,2.514,211.289143,8.106,2010.1,2010.2,2010.1
3,1,2010-02-26,1409727.59,0,46.63,2.561,211.319643,8.106,2010.1,2010.2,2010.1
4,1,2010-03-05,1554806.68,0,46.50,2.625,211.350143,8.106,2010.1,2010.3,2010.1


In [13]:
data1 = data[['Store','Temperature','Fuel_Price','CPI','Unemployment','Weekly_Sales']].copy()
data1 = data1[data1.Store.eq(1)]
data1.head()

,Store,Temperature,Fuel_Price,CPI,Unemployment,Weekly_Sales
0,1,42.31,2.572,211.096358,8.106,1643690.90
1,1,38.51,2.548,211.242170,8.106,1641957.44
2,1,39.93,2.514,211.289143,8.106,1611968.17
3,1,46.63,2.561,211.319643,8.106,1409727.59
4,1,46.50,2.625,211.350143,8.106,1554806.68


In [14]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
y_output = data1.iloc[:,5:6]
x_input = data1.iloc[:,2:5]

In [15]:
from sklearn.model_selection import train_test_split
x_train , x_test , y_train, y_test = train_test_split(x_input,y_output,test_size = 0.2,random_state=10)
print(x_train.shape , y_train.shape , x_test.shape , y_test.shape)

(114, 3) (114, 1) (29, 3) (29, 1)


In [16]:
linreg.fit(x_train,y_train)
y_pred = linreg.predict(x_test)

In [17]:
from sklearn.metrics import mean_squared_error
RMSE = np.sqrt(mean_squared_error(y_test,y_pred))
RMSE

143314.79141759968

In [18]:
accuracy = linreg.score(x_test,y_test)
accuracy

0.15819466876839283